In [ ]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

### **Connect DB**

In [ ]:
# Database connection parameters
dbname = 'news_articles'
user = 'postgres'
password = 'pass123'
host = 'localhost'
port = '5432'

# Establishing the connection
connection = psycopg2.connect(
    dbname=dbname,
    user=user,
    password=password,
    host=host,
    port=port
)

# Creating a cursor object
cursor = connection.cursor()

# Executing a SQL query to check connection
cursor.execute('SELECT version();')
result = cursor.fetchone()
print(f"PostgreSQL version: {result}")



### Creating the Schema and Table with IF NOT EXISTS

In [ ]:

create_schema_query = "CREATE SCHEMA IF NOT EXISTS sentiment_analysis;"
cursor.execute(create_schema_query)
connection.commit()

create_table_query = """
CREATE TABLE IF NOT EXISTS news.articles (
    article_id SERIAL PRIMARY KEY,
    title TEXT NOT NULL,
    description TEXT,
    content TEXT,
    category VARCHAR(255),
    title_sentiment VARCHAR(50),
    title_sentiment_score FLOAT,
    standardized_sentiment_score FLOAT,
    title_keywords TEXT[],
    content_keywords TEXT[],
    title_content_similarity FLOAT,
    title_content_cosine_similarity FLOAT,
    dominant_topic VARCHAR(255),
    event_cluster INT
);
"""
cursor.execute(create_table_query)
connection.commit()



### Inserting Data

In [ ]:

insert_query = """
INSERT INTO news.articles (
    title, description, content, category, title_sentiment, title_sentiment_score,
    standardized_sentiment_score, title_keywords, content_keywords,
    title_content_similarity, title_content_cosine_similarity, dominant_topic, event_cluster
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
article_data = (
    'Sample Title', 'Sample Description', 'Sample Content', 'Sample Category',
    'positive', 0.9, 0.85, ['keyword1', 'keyword2'], ['keywordA', 'keywordB'],
    0.75, 0.65, 'Topic A', 1
)
cursor.execute(insert_query, article_data)
connection.commit()



### Updating Data

In [ ]:

update_query = """
UPDATE news.articles
SET title_sentiment_score = %s
WHERE article_id = %s
"""
update_data = (0.95, 1)  # Example values
cursor.execute(update_query, update_data)
connection.commit()



### View Records

In [ ]:

select_query = "SELECT * FROM news.articles;"
import pprint
cursor.execute(select_query)

rows = cursor.fetchall()
for row in rows:
    pprint.pprint(row)



### DataFrame to SQL

In [ ]:
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')

# Example DataFrame
df = pd.DataFrame({
    'title': ['Example Title 1', 'Example Title 2'],
    'description': ['Example Description 1', 'Example Description 2'],
    'content': ['Example Content 1', 'Example Content 2'],
    'category': ['Category 1', 'Category 2'],
    'title_sentiment': ['positive', 'neutral'],
    'title_sentiment_score': [0.9, 0.5],
    'standardized_sentiment_score': [0.85, 0.55],
    'title_keywords': [['keyword1', 'keyword2'], ['keyword3', 'keyword4']],
    'content_keywords': [['keywordA', 'keywordB'], ['keywordC', 'keywordD']],
    'title_content_similarity': [0.75, 0.60],
    'title_content_cosine_similarity': [0.65, 0.55],
    'dominant_topic': ['Topic A', 'Topic B'],
    'event_cluster': [1, 2]
})
df.to_sql('articles', engine, schema='sentiment_analysis', if_exists='replace', index=False)

# Closing the cursor and connection
cursor.close()
connection.close()